In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import Entrez
import re
import os
from Bio import Entrez, SeqIO

# Carregar pacotes necessários

# Configurar o e-mail para acessar o NCBI
Entrez.email = "nathanpaivab@gmail.com"

In [ ]:


def search_genes_ncbi_range(start_id, end_id):
    """
    Pesquisa genes no NCBI a partir de um intervalo de códigos de identificação de genes.

    Parameters:
    start_id (str): Código de identificação do gene inicial no NCBI.
    end_id (str): Código de identificação do gene final no NCBI.

    Returns:
    list: Lista de SeqRecord contendo as sequências dos genes.
    """
    # Extrair prefixo e números dos IDs
    prefix = re.match(r"([A-Za-z]+)", start_id).group(1)
    start_num = int(re.match(r"[A-Za-z]+(\d+)", start_id).group(1))
    end_num = int(re.match(r"[A-Za-z]+(\d+)", end_id).group(1))

    gene_ids = [f"{prefix}{i:06d}" for i in range(start_num, end_num + 1)]

    gene_records = []
    for gene_id in gene_ids:
        handle = Entrez.efetch(db="nucleotide", id=gene_id, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()
        gene_records.append(record)
    return gene_records


# Exemplo de uso
start_id = "EU017577"
end_id = "EU018044"


gene_records = search_genes_ncbi_range(start_id, end_id)

# Criar pasta para armazenar as sequências
output_dir = f"{start_id}_{end_id}_sequences"
os.makedirs(output_dir, exist_ok=True)

for record in gene_records:
    # Extrair o nome da espécie e do gene do GenBank
    species_name = record.annotations.get("organism", "Unknown_species").replace(" ", "_")
    gene_name = "Unknown_gene"
    for feature in record.features:
        if feature.type == "CDS" and "gene" in feature.qualifiers:
            gene_name = feature.qualifiers["gene"][0]
            break

    # Salvar cada sequência em um arquivo separado com o formato especificado
    output_path = os.path.join(output_dir, f"{record.id}_{species_name}_{gene_name}.fasta")
    SeqIO.write(record, output_path, "fasta")
    print(f"Salvo: {output_path}")


Salvo: EU017577_EU018044_sequences/EU017577.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017578.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017579.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017580.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017581.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017582.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017583.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017584.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017585.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017586.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017587.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017588.1_Proceratophrys_boiei_cytb.fasta
Salvo: EU017577_EU018044_sequences/EU017589.1_Proceratophrys_boi

In [3]:
#concatenar sequências de genes


from collections import defaultdict

# Diretório onde as sequências estão armazenadas
input_dir = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences"

# Dicionário para armazenar sequências por gene
gene_sequences = defaultdict(list)

# Iterar sobre os arquivos no diretório
for filename in os.listdir(input_dir):
    if filename.endswith(".fasta"):
        filepath = os.path.join(input_dir, filename)
        record = SeqIO.read(filepath, "fasta")
        gene_name = filename.split('_')[-1].replace(".fasta", "")
        gene_sequences[gene_name].append(record)

# Concatenar sequências de cada gene em um único arquivo
for gene_name, records in gene_sequences.items():
    output_path = os.path.join(input_dir, f"{gene_name}_concatenated.fasta")
    SeqIO.write(records, output_path, "fasta")
    print(f"Salvo: {output_path}")

Salvo: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_concatenated.fasta
Salvo: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_concatenated.fasta


In [4]:
#concatenar sequências de genes


from collections import defaultdict

# Diretório onde as sequências estão armazenadas
input_dir = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences"

# Dicionário para armazenar sequências por gene
gene_sequences = defaultdict(list)

# Iterar sobre os arquivos no diretório
for filename in os.listdir(input_dir):
    if filename.endswith(".fasta"):
        filepath = os.path.join(input_dir, filename)
        record = SeqIO.read(filepath, "fasta")
        gene_name = filename.split('_')[-1].replace(".fasta", "")
        gene_sequences[gene_name].append(record)

# Concatenar sequências de cada gene em um único arquivo
for gene_name, records in gene_sequences.items():
    output_path = os.path.join(input_dir, f"{gene_name}_concatenated.fasta")
    SeqIO.write(records, output_path, "fasta")
    print(f"Salvo: {output_path}")

Salvo: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_concatenated.fasta


In [5]:
#filtrar sequências de NAD2 apenas de Proceratophrys boiei

from Bio import SeqIO

# Caminho do arquivo concatenado
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_concatenated.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei.fasta"

# Ler as sequências do arquivo concatenado
records = list(SeqIO.parse(input_file, "fasta"))

# Filtrar sequências que pertencem à espécie Proceratophrys boiei
filtered_sequences = [record for record in records if "Proceratophrys boiei" in record.description]

# Salvar as sequências filtradas em um novo arquivo
SeqIO.write(filtered_sequences, output_file, "fasta")

print(f"Sequências filtradas salvas em: {output_file}")

Sequências filtradas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei.fasta


In [6]:
#filtrar sequências de cytb apenas de Proceratophrys boiei
from Bio import SeqIO

# Caminho do arquivo concatenado
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_concatenated.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei.fasta"

# Ler as sequências do arquivo concatenado
records = list(SeqIO.parse(input_file, "fasta"))

# Filtrar sequências que pertencem à espécie Proceratophrys boiei
filtered_sequences = [record for record in records if "Proceratophrys boiei" in record.description]

# Salvar as sequências filtradas em um novo arquivo
SeqIO.write(filtered_sequences, output_file, "fasta")

print(f"Sequências filtradas salvas em: {output_file}")

Sequências filtradas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei.fasta


In [7]:
#filtrar sequências de c-myc apenas de Proceratophrys boiei
from Bio import SeqIO

# Caminho do arquivo concatenado
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_concatenated.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei.fasta"

# Ler as sequências do arquivo concatenado
records = list(SeqIO.parse(input_file, "fasta"))

# Filtrar sequências que pertencem à espécie Proceratophrys boiei
filtered_sequences = [record for record in records if "Proceratophrys boiei" in record.description]

# Salvar as sequências filtradas em um novo arquivo
SeqIO.write(filtered_sequences, output_file, "fasta")

print(f"Sequências filtradas salvas em: {output_file}")

Sequências filtradas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei.fasta


In [8]:
#renomear as sequências de ND2 de Proceratophrys boiei para que fique apenas o código e o quinto termo do cabeçalho

from Bio import SeqIO

# Caminho do arquivo de entrada e saída
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_renamed.fasta"

# Ler as sequências do arquivo de entrada
records = list(SeqIO.parse(input_file, "fasta"))

# Renomear os cabeçalhos das sequências
for record in records:
    description_parts = record.description.split()
    sequence_id = description_parts[0]
    fifth_term = description_parts[4] if len(description_parts) > 4 else ""
    new_description = f"{sequence_id} {fifth_term}"
    record.description = new_description
    record.id = sequence_id  # Manter apenas o ID da sequência

# Salvar as sequências com os novos cabeçalhos
SeqIO.write(records, output_file, "fasta")

print(f"Sequências renomeadas salvas em: {output_file}")

Sequências renomeadas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_renamed.fasta


In [9]:
#renomear as sequências de cytb de Proceratophrys boiei para que fique apenas o código e o quinto termo do cabeçalho

from Bio import SeqIO

# Caminho do arquivo de entrada e saída
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_renamed.fasta"

# Ler as sequências do arquivo de entrada
records = list(SeqIO.parse(input_file, "fasta"))

# Renomear os cabeçalhos das sequências
for record in records:
    description_parts = record.description.split()
    sequence_id = description_parts[0]
    fifth_term = description_parts[4] if len(description_parts) > 4 else ""
    new_description = f"{sequence_id} {fifth_term}"
    record.description = new_description
    record.id = sequence_id  # Manter apenas o ID da sequência

# Salvar as sequências com os novos cabeçalhos
SeqIO.write(records, output_file, "fasta")

print(f"Sequências renomeadas salvas em: {output_file}")

Sequências renomeadas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_renamed.fasta


In [10]:
#renomear as sequências de ND2 de Proceratophrys boiei para que fique apenas o código e o quinto termo do cabeçalho

from Bio import SeqIO

# Caminho do arquivo de entrada e saída
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_renamed.fasta"

# Ler as sequências do arquivo de entrada
records = list(SeqIO.parse(input_file, "fasta"))

# Renomear os cabeçalhos das sequências
for record in records:
    description_parts = record.description.split()
    sequence_id = description_parts[0]
    fifth_term = description_parts[4] if len(description_parts) > 4 else ""
    new_description = f"{sequence_id} {fifth_term}"
    record.description = new_description
    record.id = sequence_id  # Manter apenas o ID da sequência

# Salvar as sequências com os novos cabeçalhos
SeqIO.write(records, output_file, "fasta")

print(f"Sequências renomeadas salvas em: {output_file}")

Sequências renomeadas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_renamed.fasta


In [ ]:
#caso queira as sequências de Proceratophrys boiei apenas com o número do voucher 

In [11]:
from Bio import SeqIO

# Caminho do arquivo de entrada e saída
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_renamed.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_renamed_second_term.fasta"

# Ler as sequências do arquivo de entrada
records = list(SeqIO.parse(input_file, "fasta"))

# Renomear os cabeçalhos das sequências
for record in records:
    description_parts = record.description.split()
    second_term = description_parts[1] if len(description_parts) > 1 else ""
    new_description = f"{second_term}"
    record.description = new_description
    record.id = second_term  # Manter apenas o segundo termo como ID da sequência

# Salvar as sequências com os novos cabeçalhos
SeqIO.write(records, output_file, "fasta")

print(f"Sequências renomeadas salvas em: {output_file}")

Sequências renomeadas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/cytb_Proceratophrys_boiei_renamed_second_term.fasta


In [12]:
from Bio import SeqIO

# Caminho do arquivo de entrada e saída
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_renamed.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_renamed_second_term.fasta"

# Ler as sequências do arquivo de entrada
records = list(SeqIO.parse(input_file, "fasta"))

# Renomear os cabeçalhos das sequências
for record in records:
    description_parts = record.description.split()
    second_term = description_parts[1] if len(description_parts) > 1 else ""
    new_description = f"{second_term}"
    record.description = new_description
    record.id = second_term  # Manter apenas o segundo termo como ID da sequência

# Salvar as sequências com os novos cabeçalhos
SeqIO.write(records, output_file, "fasta")

print(f"Sequências renomeadas salvas em: {output_file}")

Sequências renomeadas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU017577_EU018044_sequences/ND2_Proceratophrys_boiei_renamed_second_term.fasta


In [13]:
from Bio import SeqIO

# Caminho do arquivo de entrada e saída
input_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_renamed.fasta"
output_file = "/home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_renamed_second_term.fasta"

# Ler as sequências do arquivo de entrada
records = list(SeqIO.parse(input_file, "fasta"))

# Renomear os cabeçalhos das sequências
for record in records:
    description_parts = record.description.split()
    second_term = description_parts[1] if len(description_parts) > 1 else ""
    new_description = f"{second_term}"
    record.description = new_description
    record.id = second_term  # Manter apenas o segundo termo como ID da sequência

# Salvar as sequências com os novos cabeçalhos
SeqIO.write(records, output_file, "fasta")

print(f"Sequências renomeadas salvas em: {output_file}")

Sequências renomeadas salvas em: /home/nathan/Documents/Doutorado_diversidade_genética/Filogeografia/EU025551_EU025679_sequences/c-myc_Proceratophrys_boiei_renamed_second_term.fasta
